In [1]:
import cv2
import numpy as np
import mediapipe as mp

In [2]:
mp_face_mesh = mp.solutions.face_mesh

face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    static_image_mode=True,
    refine_landmarks=True,
    min_detection_confidence=0.8,
    min_tracking_confidence=0.8
)

In [3]:
def get_landmarks(image):
    landmarks = {}
    h,w = image.shape[:2]
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    face_mesh_result = face_mesh.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    if face_mesh_result.multi_face_landmarks:
        for i, landmark in enumerate(face_mesh_result.multi_face_landmarks[0].landmark): 
            x = landmark.x
            y = landmark.y
            relative_x = int(x * w)
            relative_y = int(y * h)
            landmarks[i+1] = (relative_x, relative_y)
    return landmarks

In [4]:
def get_coordinates(landmarks):
    coordinates = {
        "eye_left": [landmarks[30], [landmarks[158][0], landmarks[145][1]]],
        "eye_right": [landmarks[287], [landmarks[260][0], landmarks[381][1]]],
        # "shade": [landmarks[71], (landmarks[301][0], landmarks[301][1] + (landmarks[119][1] - landmarks[301][1]))]
        "shade": [(landmarks[140][0], landmarks[72][1]), landmarks[117], landmarks[346], landmarks[390]]
    }
    return coordinates

In [5]:
def draw_coordinates(image, points, color = [150, 0, 200]):
    neighbor_vector = [(0,0), (1,0), (0,1), (-1,0), (0, -1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
    for pt in points:
        image[pt[1]][pt[0]] = color
    return image

In [20]:
import math
def get_angle(coordinates):
    height = (coordinates[1][1] - coordinates[2][1])
    base = (coordinates[1][0] - coordinates[2][0])
    
    angle = math.atan(height/base) * 180/math.pi
    return angle
    

In [25]:
def rotate_image(im, angle):
    imHeight, imWidth = im.shape[0], im.shape[1]
    centreX, centreY = imWidth//2, imHeight//2
        
    rotationMat = cv2.getRotationMatrix2D(
    center=(centreX, centreY),
    angle=20,
    scale=1
    )
    
    cos = np.abs(rotationMat[0][0])
    sin = np.abs(rotationMat[1][0])
    
    newWidth = int((imHeight * sin) + (imWidth * cos))
    newHeight = int((imHeight * cos) + (imWidth * sin))
    
    rotationMat[0][2] = 0
    rotationMat[1][2] += newHeight/2 - centreY
    
    dst_mat = np.zeros((newHeight, newWidth, 4), np.uint8)
    rotatedMat = cv2.warpAffine(
        im,
        rotationMat,
        (newWidth, newHeight),
        dst_mat
    )
    
    return dst_mat, (newHeight, newWidth)

In [24]:
cam = cv2.VideoCapture(0)

while True:
    ret, frame = cam.read()
    frame = frame[:,::-1]
    
    if ret:
        landmarks = get_landmarks(frame)
        coordinates = get_coordinates(landmarks)
        image = draw_coordinates(frame, coordinates['shade'])
        
        image = np.ascontiguousarray(image, dtype=np.uint8)
        angle = get_angle(coordinates['shade'])
        
        cv2.putText(image, str(angle), (50, 50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, [255, 255, 255], 1)
        
        cv2.imshow("im", image)
        if cv2.waitKey(5) & 0xFF == 27:
            cv2.destroyAllWindows()
            cam.release()
            break